In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from IPython.display import clear_output
from tensorflow.keras import backend as K
from skimage.transform import resize

import keras.layers as kl
import tensorflow as tf

In [2]:
traindf = pd.read_csv('../input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip')

In [3]:
import zipfile

with zipfile.ZipFile("/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip","r") as z:
    z.extractall(".")

In [4]:
def append_ext(fn):
    return fn + ".jpg"

traindf["id"] = traindf['GalaxyID'].astype(str).apply(append_ext)

In [5]:
#def get_image(path):
#    image = plt.imread(path)
#    im_size_x = image.shape[0]
#    im_size_y = image.shape[1]
#    print(im_size_x,im_size_y)
#    cropx = im_size_x // 4
#    cropy = im_size_y // 4
#    print(cropx, cropy)
#    image = image[cropx:im_size_x-cropx, cropy:im_size_y-cropy]
#    print(image.shape)
#    return image

#path = image_dir+'806578.jpg'
#tmp_im = get_image(path)
#print(tmp_im.shape)

In [6]:
def crop_image(image):
  #Image - numpy array of rank 3
  #Crop image to half its size, preserving the center
  #resize image to shape
  im_size_x = image.shape[0]
  im_size_y = image.shape[1]
  cropx = im_size_x // 4
  cropy = im_size_y // 4
  image = image[cropx:im_size_x-cropx, cropy:im_size_y-cropy]

  re_shape = [212,212]
  image = resize(image, re_shape)
  return image

datagen = ImageDataGenerator(
    fill_mode='constant',
    cval=0,
    rescale=1. / 255,
    #rotation_range=90,
    #width_shift_range=0.1,
    #height_shift_range=0.1,
    #horizontal_flip=True,
    #vertical_flip=True,
    preprocessing_function=crop_image,
    validation_split=0.2)

image_dir = './images_training_rev1/'
classes = ['Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1','Class3.2', 
           'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3','Class5.4', 'Class6.1', 
           'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3','Class8.1', 'Class8.2', 'Class8.3', 
           'Class8.4', 'Class8.5', 'Class8.6', 'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 
           'Class10.1', 'Class10.2', 'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 
           'Class11.4', 'Class11.5', 'Class11.6']

train_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=image_dir,
    x_col="id",
    y_col=classes,
    subset="training",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="raw",
    target_size=(212,212))

valid_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=image_dir,
    x_col="id",
    y_col=classes,
    subset="validation",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="raw",
    target_size=(212,212))

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

Found 49263 validated image filenames.
Found 12315 validated image filenames.


In [7]:
print(train_generator[0][0][0].shape)

(212, 212, 3)


In [8]:
img_shape = (212,212, 3)
resnet_model = ResNet50(include_top=False, input_shape=img_shape)

94773248/94765736 [==============================] - 1s 0us/step


In [9]:
# Flatten output of last layer before adding output layer (Dense layer)
x = kl.Flatten()(resnet_model.output)
# Add output layer (number of outputs = 37)
x = kl.Dense(len(classes), activation='sigmoid')(x)
# Load the modified model
model = tf.keras.Model(inputs=resnet_model.input, outputs=x)

In [10]:
for layer in model.layers:
    layer.trainable = True

In [11]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=[root_mean_squared_error])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 212, 212, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 218, 218, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 106, 106, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 106, 106, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [12]:
import time

start_time = time.time()

data_dir = './'

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
 
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

early_stopping = EarlyStopping(
    monitor='val_loss', patience=4, verbose=1, mode='auto')

history = LossHistory()

checkpointer = ModelCheckpoint(
    filepath=data_dir+'weights_full.hdf5', verbose=2, save_best_only=True)

hist = model.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=valid_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=15,
    callbacks=[history, checkpointer, early_stopping])

elapsed_time = time.time() - start_time
print("Elapsed time: {}", elapsed_time)

Epoch 1/15
769/769 [==============================] - ETA: 0s - loss: 0.2593 - root_mean_squared_error: 0.1130
Epoch 00001: val_loss improved from inf to 0.30665, saving model to ./weights_full.hdf5
769/769 [==============================] - 628s 817ms/step - loss: 0.2593 - root_mean_squared_error: 0.1130 - val_loss: 0.3066 - val_root_mean_squared_error: 0.1683
Epoch 2/15
769/769 [==============================] - ETA: 0s - loss: 0.2377 - root_mean_squared_error: 0.0940
Epoch 00002: val_loss improved from 0.30665 to 0.23925, saving model to ./weights_full.hdf5
769/769 [==============================] - 632s 822ms/step - loss: 0.2377 - root_mean_squared_error: 0.0940 - val_loss: 0.2392 - val_root_mean_squared_error: 0.0956
Epoch 3/15
769/769 [==============================] - ETA: 0s - loss: 0.2341 - root_mean_squared_error: 0.0881
Epoch 00003: val_loss improved from 0.23925 to 0.23760, saving model to ./weights_full.hdf5
769/769 [==============================] - 625s 813ms/step - loss

AttributeError: 'str' object has no attribute 'elapsed_time'

In [ ]:
model.save("my_h5_model.h5")

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(hist.epoch, hist.history['loss'], label='Training Loss')
plt.plot(
    hist.epoch, hist.history['val_loss'], label='Validation', linestyle='--')
plt.xlabel("Epochs")
plt.ylabel("RMSE")
plt.legend()
plt.show()